# Metric Extraction

### Requirements

1. Spacy: `conda install -c conda-forge spacy`
2. English corpus for spacy: `python -m spacy download en_core_web_sm`
3. text2num: `pip install text2num`

### Outline

In [ ]:
import logging
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

## Data Source Selection

### Option 1: Use Search Terms 

1. Define the `SEARCH_QUERY` variable in the cell below with a list of keywords that describe the science branch of your interest.
2. Set `USE_PUBMED_IMPORT` to `False`.

### Option 2: Import Search Results from Pubmed 

1. Set `USE_PUBMED_IMPORT` to `True`.
2. Search for terms of interest on the Pubmed website. Use Save with Selection=All Results and Format=PMID to obtain a .txt file with PMIDs of all papers that were found.
3. Use widget below to upload a file with IDs from Pubmed.

In [ ]:
# Option 1 - Use Search Terms
SEARCH_QUERY = 'zebrafish toxicity'

# Option 2 - Import Search Results from Pubmed
USE_PUBMED_IMPORT = False

## Import PMIDs from Pubmed

In [ ]:
import ipywidgets as widgets

from IPython.display import display

if USE_PUBMED_IMPORT:
    w = widgets.FileUpload(accept='.txt', multiple=False)
    display(w)

## Load Publication Data

In [ ]:
from bokeh.plotting import show, output_notebook
from matplotlib import pyplot as plt

from pysrc.papers.config import PubtrendsConfig
from pysrc.papers.progress import Progress
from pysrc.papers.db.loaders import Loaders
from pysrc.papers.analyzer import KeyPaperAnalyzer
from pysrc.papers.analyzer_experimental import ExperimentalAnalyzer
from pysrc.papers.plot.plotter_experimental import ExperimentalPlotter
from pysrc.papers.utils import SORT_MOST_CITED, SORT_MOST_RELEVANT, SORT_MOST_RECENT, cut_authors_list

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')
output_notebook()
%matplotlib inline

SEARCH_SORT = SORT_MOST_CITED
SEARCH_PAPERS = 500

In [ ]:
config = PubtrendsConfig(test=False)
loader = Loaders.get_loader('Pubmed', config)
try:
    if USE_PUBMED_IMPORT:
        logging.info('Loading from file')
        filename = list(w.value.keys())[0]
        ids = [int(chunk) for chunk in w.value[filename]['content'].decode('utf-8').split('\r\n')]
    else:
        logging.info('Searching')
        ids = loader.search(SEARCH_QUERY, limit=SEARCH_PAPERS, sort=SEARCH_SORT, noreviews=True)
        ids = KeyPaperAnalyzer(loader, config).expand_ids(
            ids, limit=int(1.2 * SEARCH_PAPERS), keep_keywords=True, steps=2
        )
    pub_df = loader.load_publications(ids)
finally:
    loader.close_connection()
    logging.info('Done')

## Core Function for Metric Extraction

In [ ]:
import re
import spacy

from collections import Counter
from spacy import displacy
from text_to_num import alpha2digit
from nltk.stem import WordNetLemmatizer

NUMBER = re.compile(r'-?[\d]+([\.,][\d]+)?([eE][+-]?\d+)?')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
lemmatizer = WordNetLemmatizer()

def process_candidate(metrics, token, value, idx):
    tt = token.text
    if tt != '%' and token.pos_ in set(['NOUN', 'PROPN']):
        tt = lemmatizer.lemmatize(tt)
        if tt not in metrics:
            metrics[tt] = []
        metrics[tt].append((value, idx))
        return True
    return False

def process_number(token, value, idx, metrics):
    logging.info(f'Number: {value}')
    if token.head.pos_ == 'NUM':
        tht = token.head.text
        if tht == 'hundred':
            value *= 100
        elif tht == 'thousand':
            value *= 1000
        elif tht == 'million':
            value *= 1000000
        elif tht == 'billion':
            value *= 1000000000
        logging.info(f'Value adjusted: {value}')
        token = next(token.ancestors)
        
    # Analyze children and siblings, then ancestors if first was not enough
    # TODO: is there a better way?
    # TODO: use close nouns as a fallback when it is hard to find a dependency?
    # TODO: expand nouns with adjectives or other nouns? (rate -> information transfer rate)
    logging.info(f'Token children: {",".join(t.text for t in token.children)}')
    children_matched = False
    for t in token.children:
        if process_candidate(metrics, t, value, idx):
            logging.info(f'Child term: {t.text}')
            children_matched = True
        if children_matched:
            return
    logging.info('Head with children: '
                 f'{token.head.text} | {",".join(t.text for t in token.head.children)}')  
    if process_candidate(metrics, token.head, value, idx):
        logging.info(f'Head term: {token.head.text}')
        return
    
    head_children_matched = False
    for t in token.head.children:        
        if process_candidate(metrics, t, value, idx):
            logging.info(f'Child of head term: {t.text}')
            head_children_matched = True
    if head_children_matched:
        return

    logging.info(f'Token anscestors: {",".join(t.text for t in token.ancestors)}')
    for t in token.ancestors:
        if process_candidate(metrics, t, value, idx):
            logging.info(f'Ancestor: {t.text}')
            return

def extract_metrics(text, visualize_dependencies=False):
    """
    Parses abstract and returns a dict of numbers with nouns that could be suitable as a metric.
    :return list of tuples (sentence, [metrics]), where metrics is a list of tuples (number, [nouns], sentence_number)
    """   
    metrics = {}
    sentences = {}
    # Convert textual numbers to digits (three -> 3)
    text = alpha2digit(text.lower(), 'en', relaxed=True)
    # Convect 10th -> 10
    text = re.sub(r"([\d]+)th", r"\g<1>", text)
    # Split text into sentences and find numbers in sentences
    doc = spacy_en(text)
    for idx, sent in enumerate(doc.sents):
        logging.info('###')
        logging.info(sent.text)        
        sentences[idx] = sent.text
        for token in sent:
#             print(token.text, token.pos_, list(token.ancestors))
            if NUMBER.fullmatch(token.text):
                tt = token.text.replace(',', '')  # TODO: can fail in case of different locale
                value = float(tt) if '.' in tt else int(tt)
                process_number(token, value, idx, metrics)            
        if visualize_dependencies:
            displacy.render(sent, style="dep", jupyter=True)
    return metrics, sentences

### Demo `extract_metrics`

In [ ]:
TEXT = """
No specific antiviral drug has been proven effective for treatment of patients with 
 severe coronavirus disease 2019 (COVID-19).

Over brief training periods of 3-24 min, four patients then used these signals 
 to master closed-loop control and to achieve success rates of 74-100% in a 
 one-dimensional binary task.

We performed a weighted multivariate analysis of urinary creatinine concentrations in 
 22,245 participants of the third National Health and Nutrition Examination Survey (1988-1994) 
 and established reference ranges (10th-90th percentiles) for each demographic and age category.

Longitudinal descriptive analyses of the 1032 participants in the 1991-2007 
 National Institute of Child Health and Human Development Study of Early Child Care and 
 Youth Development birth cohort from 10 study sites who had accelerometer-determined minutes of MVPA 
 at ages 9 (year 2000), 11 (2002), 12 (2003), and 15 (2006) years.
 
Hookworm infection occurs in almost half of ssa's poorest people, including 40-50 million school-aged 
children and 70 million pregnant women in whom it is a leading cause of anemia.

For the 2 most mutagenic regimens: 4 x 1 hr in 3 mm enu and 6 x 1', 5: 'hr in 3 mm enu'.
"""

metrics, _ = extract_metrics(TEXT, visualize_dependencies=True)
for word, occasions in metrics.items():
    print(f"{word}: {occasions}")

## Let's apply metric extraction to Pubmed papers!

### Exclude papers with unwanted terms

In [ ]:
EXCLUDE = ['\becog', 'electrocorticograph', '\bhybrid', 'fNIRS', '\bSSVEP']

if len(EXCLUDE) > 0:
    EXCLUDE_REGEX = '|'.join(EXCLUDE)
    EXCLUDE_MASK = np.logical_not(pub_df.abstract.str.contains(EXCLUDE_REGEX, flags=re.IGNORECASE, regex=True))
    processed_pub_df = pub_df[EXCLUDE_MASK]
else:
    processed_pub_df = pub_df

In [ ]:
original_ids = set(pub_df['id'])
processed_ids = set(processed_pub_df['id'])
diff = original_ids - processed_ids

See the list of excluded papers below.

In [ ]:
pub_df[pub_df['id'].isin(diff)][['id', 'abstract']]

## Run Metric Extraction

In [ ]:
from tqdm.auto import tqdm

# Slow, currently moved out of the class to speed up fixing & rerunning the code of MetricExtractor
metrics_data = []
for _, data in tqdm(processed_pub_df.iterrows()):
    paper_metrics_data = [data['id'], *extract_metrics(data['abstract'])]
    metrics_data.append(paper_metrics_data)

In [ ]:
class MetricExtractor:
    def __init__(self, metrics_data):
        self.metrics_df = pd.DataFrame(metrics_data, columns=['ID', 'Metrics', 'Sentences'])
        
    def get_top_metrics(self, number=20):
        metrics_counter = Counter()
        for metric_dict in self.metrics_df['Metrics']:
            for metric, occasions in metric_dict.items():
                metrics_counter[metric] += len(occasions)
        return metrics_counter.most_common(number)
    
    def get_metric_values(self, *metrics, min_value=None, max_value=None, detailed=False):
        values = []
        for _, data in self.metrics_df.iterrows():
            metric_dict = data['Metrics']
            sentences = data['Sentences']
        
            for metric in metrics:
                if metric in metric_dict:
                    for value, sentence_number in metric_dict[metric]:
                        if min_value and value < min_value or max_value and value > max_value:
                            continue
                        if detailed:
                            sentence = sentences[sentence_number]
                            values.append([data['ID'], value, sentence])
                        else:
                            values.append(value)
        if detailed:
            return pd.DataFrame(values, columns=['PMID', ', '.join(metrics), 'Sentence'])
        return values
    
    def filter_papers(self, metrics):
        """
        :param metrics - list of tuples ([list of keywords], min_value, max_value)
               e.g. (['subjects', 'participants'], 5, None)
        :return list of PMIDs
        """
        selection = []
        for _, data in self.metrics_df.iterrows():
            suitable = True
            metric_dict = data['Metrics']
            
            for metric in metrics:
                metric_suitable = False
                words, min_value, max_value = metric
                
                for word in words:
                    if word in metric_dict:
                        for value, _ in metric_dict[word]:
                            if min_value and value < min_value or max_value and value > max_value:
                                continue
                            metric_suitable = True
                    if metric_suitable:
                        break
                
                suitable &= metric_suitable
                    
            if suitable:
                selection.append(data['ID'])
        return selection

In [ ]:
me = MetricExtractor(metrics_data)

### See an example of extraction result below.

In [ ]:
me.metrics_df.head(3)

In [ ]:
result = pd.merge(left=me.metrics_df, left_on='ID', right=pub_df[['id', 'title']], right_on='id')
result = result[['ID', 'title', 'Metrics', 'Sentences']]
result.to_csv(f'metrics_{SEARCH_QUERY.replace(" ", "_").lower()}.csv', sep=',', index=False)

### Most Frequent Metrics

In [ ]:
print(me.get_top_metrics(50))

### Select a metric to show distribution and abstract fragments where it was mentioned.

Currently synonyms are not processed, so feel free to use a tuple of words that correspond to the same metric.

In [ ]:
METRIC = ('subject', 'patient', 'role', 'participant', 'volunteer', 'people', 'donor', 
          'man', 'woman', 'male', 'female')
METRIC_TEXT = ', '.join(METRIC)

### Distribution of selected `METRIC`

In [ ]:
import matplotlib.pyplot as plt

num_subjects = me.get_metric_values(*METRIC)
plt.hist(num_subjects)
plt.xlabel(METRIC_TEXT)
plt.ylabel('Number of papers')

### Example Mentions 

In [ ]:
me.get_metric_values(*METRIC, detailed=True).sort_values(METRIC_TEXT, ascending=False).head(20)

### Filter papers based on desired metric values

In [ ]:
pub_df[pub_df.id.isin(selection)][['id', 'abstract']]

## Work with synonims

In [ ]:
from itertools import chain
from nltk.corpus import wordnet

synonyms = wordnet.synsets('patient')
set(chain.from_iterable([word.lemma_names() for word in synonyms]))

## Development

Other libraries for extraction of metrics/quantities: quantulum, grobid-quantities.

Dependencies:
  * `pip install quantulum3`

In [ ]:
from quantulum3 import parser
parser.parse("""No specific antiviral drug has been proven effective for treatment of patients with 
 severe coronavirus disease 2019 (COVID-19).""")